# Visualization libraries

Use tensorboard to graphically visualize what's going on

Simple example below using tensorboardX

In [1]:
import math
from tensorboardX import SummaryWriter

writer = SummaryWriter()
funcs = {
    'sin': math.sin,
    'cos': math.cos,
    'tan': math.tan
}

for angle in range(-360, 360):
    angle_rad = angle * math.pi / 180
    for name, fun in funcs.items():
        val = fun(angle_rad)
        writer.add_scalar(name, val, angle)
    writer.close()

## Advanced example using GANs

Define a gym game and use GANs

In [6]:
import random
import argparse
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
from tensorboardX import SummaryWriter

import torchvision.utils as vutils

import gym
import gym.spaces

import numpy as np

log = gym.logger
log.set_level(gym.logger.INFO)

BATCH_SIZE = 16


class InputWrapper(gym.ObservationWrapper):
    def __init__(self, *args):
        super(InputWrapper, self).__init__(*args)
        assert isinstance(self.observation_space, gym.spaces.Box)
        old_space = self.observation_space
        self.observation_space = gym.spaces.Box(
            self.observation(old_space.low),
            self.observation(old_space.high),
            dtype=np.float32
        )
        
    def observation(self, observation):
        new_obs = cv2.resize(
            observation, (IMAGE_SIZE, IMAGE_SIZE)
        )
        new_obs = np.moveaxis(new_obs, 2, 0)
        return new_obs.astype(np.float32)
    
class Discriminator(nn.Module):
    def __init__(self, input_shape):
        super(Discriminator, self).__init__()
        # this pipe converges image into the single number
        self.conv_pipe = nn.Sequential(
            nn.Conv2d(in_channels=input_shape[0], out_channels=DISCR_FILTERS,
                      kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=DISCR_FILTERS, out_channels=DISCR_FILTERS*2,
                      kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(DISCR_FILTERS*2),
            nn.ReLU(),
            nn.Conv2d(in_channels=DISCR_FILTERS * 2, out_channels=DISCR_FILTERS * 4,
                      kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(DISCR_FILTERS * 4),
            nn.ReLU(),
            nn.Conv2d(in_channels=DISCR_FILTERS * 4, out_channels=DISCR_FILTERS * 8,
                      kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(DISCR_FILTERS * 8),
            nn.ReLU(),
            nn.Conv2d(in_channels=DISCR_FILTERS * 8, out_channels=1,
                      kernel_size=4, stride=1, padding=0),
            nn.Sigmoid()
        )

    def forward(self, x):
        conv_out = self.conv_pipe(x)
        return conv_out.view(-1, 1).squeeze(dim=1)


class Generator(nn.Module):
    def __init__(self, output_shape):
        super(Generator, self).__init__()
        # pipe deconvolves input vector into (3, 64, 64) image
        self.pipe = nn.Sequential(
            nn.ConvTranspose2d(in_channels=LATENT_VECTOR_SIZE, out_channels=GENER_FILTERS * 8,
                               kernel_size=4, stride=1, padding=0),
            nn.BatchNorm2d(GENER_FILTERS * 8),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=GENER_FILTERS * 8, out_channels=GENER_FILTERS * 4,
                               kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(GENER_FILTERS * 4),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=GENER_FILTERS * 4, out_channels=GENER_FILTERS * 2,
                               kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(GENER_FILTERS * 2),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=GENER_FILTERS * 2, out_channels=GENER_FILTERS,
                               kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(GENER_FILTERS),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=GENER_FILTERS, out_channels=output_shape[0],
                               kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        return self.pipe(x)


def iterate_batches(envs, batch_size=BATCH_SIZE):
    batch = [e.reset() for e in envs]
    env_gen = iter(lambda: random.choice(envs), None)

    while True:
        e = next(env_gen)
        obs, reward, is_done, _ = e.step(e.action_space.sample())
        if np.mean(obs) > 0.01:
            batch.append(obs)
        if len(batch) == batch_size:
            # Normalising input between -1 to 1
            batch_np = np.array(batch, dtype=np.float32)
            batch_np *= 2.0 / 255.0 - 1.0
            yield torch.tensor(batch_np)
            batch.clear()
        if is_done:
            e.reset()

In [7]:
# Run the environment

LATENT_VECTOR_SIZE = 100
DISCR_FILTERS = 4
GENER_FILTERS = 64


# dimension input image will be rescaled
IMAGE_SIZE = 64

LEARNING_RATE = 0.0001
REPORT_EVERY_ITER = 100
SAVE_IMAGE_EVERY_ITER = 1000

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'using device = {device}')
envs = [
        InputWrapper(gym.make(name))
        for name in ('Breakout-v0', 'AirRaid-v0', 'Pong-v0')
]

input_shape = envs[0].observation_space.shape

net_discr = Discriminator(input_shape=input_shape).to(device)
net_gener = Generator(output_shape=input_shape).to(device)

objective = nn.BCELoss()
gen_optimizer = optim.Adam(
    params=net_gener.parameters(), lr=LEARNING_RATE,
    betas=(0.5, 0.999))
dis_optimizer = optim.Adam(
    params=net_discr.parameters(), lr=LEARNING_RATE,
    betas=(0.5, 0.999))
writer = SummaryWriter()

gen_losses = []
dis_losses = []
iter_no = 0

true_labels_v = torch.ones(BATCH_SIZE, device=device)
fake_labels_v = torch.zeros(BATCH_SIZE, device=device)

for batch_v in iterate_batches(envs):
    # fake samples, input is 4D: batch, filters, x, y
    gen_input_v = torch.FloatTensor(
        BATCH_SIZE, LATENT_VECTOR_SIZE, 1, 1)
    gen_input_v.normal_(0, 1)
    gen_input_v = gen_input_v.to(device)
    batch_v = batch_v.to(device)
    gen_output_v = net_gener(gen_input_v)

    # train discriminator
    dis_optimizer.zero_grad()
    dis_output_true_v = net_discr(batch_v)
    dis_output_fake_v = net_discr(gen_output_v.detach())
    dis_loss = objective(dis_output_true_v, true_labels_v) + \
               objective(dis_output_fake_v, fake_labels_v)
    dis_loss.backward()
    dis_optimizer.step()
    dis_losses.append(dis_loss.item())

    # train generator
    gen_optimizer.zero_grad()
    dis_output_v = net_discr(gen_output_v)
    gen_loss_v = objective(dis_output_v, true_labels_v)
    gen_loss_v.backward()
    gen_optimizer.step()
    gen_losses.append(gen_loss_v.item())

    iter_no += 1
    if iter_no % REPORT_EVERY_ITER == 0:
        log.info("Iter %d: gen_loss=%.3e, dis_loss=%.3e",
                 iter_no, np.mean(gen_losses),
                 np.mean(dis_losses))
        writer.add_scalar(
            "gen_loss", np.mean(gen_losses), iter_no)
        writer.add_scalar(
            "dis_loss", np.mean(dis_losses), iter_no)
        gen_losses = []
        dis_losses = []
    if iter_no % SAVE_IMAGE_EVERY_ITER == 0:
        writer.add_image("fake", vutils.make_grid(
            gen_output_v.data[:64], normalize=True), iter_no)
        writer.add_image("real", vutils.make_grid(
            batch_v.data[:64], normalize=True), iter_no)

using device = cuda
INFO: Making new env: Breakout-v0
INFO: Making new env: AirRaid-v0
INFO: Making new env: Pong-v0
INFO: Iter 100: gen_loss=2.119e-01, dis_loss=2.258e+00
INFO: Iter 200: gen_loss=3.785e-01, dis_loss=1.772e+00
INFO: Iter 300: gen_loss=5.633e-01, dis_loss=1.522e+00
INFO: Iter 400: gen_loss=7.604e-01, dis_loss=1.268e+00
INFO: Iter 500: gen_loss=7.595e-01, dis_loss=1.248e+00
INFO: Iter 600: gen_loss=9.419e-01, dis_loss=1.049e+00
INFO: Iter 700: gen_loss=1.005e+00, dis_loss=9.566e-01
INFO: Iter 800: gen_loss=1.047e+00, dis_loss=9.872e-01
INFO: Iter 900: gen_loss=1.228e+00, dis_loss=8.391e-01
INFO: Iter 1000: gen_loss=1.391e+00, dis_loss=6.923e-01
INFO: Iter 1100: gen_loss=1.349e+00, dis_loss=7.082e-01
INFO: Iter 1200: gen_loss=1.537e+00, dis_loss=5.893e-01
INFO: Iter 1300: gen_loss=1.765e+00, dis_loss=4.793e-01
INFO: Iter 1400: gen_loss=1.588e+00, dis_loss=5.362e-01
INFO: Iter 1500: gen_loss=1.852e+00, dis_loss=4.522e-01
INFO: Iter 1600: gen_loss=2.113e+00, dis_loss=3.280e

INFO: Iter 14500: gen_loss=6.333e+00, dis_loss=3.058e-03
INFO: Iter 14600: gen_loss=6.898e+00, dis_loss=1.884e-03
INFO: Iter 14700: gen_loss=6.029e+00, dis_loss=6.536e-02
INFO: Iter 14800: gen_loss=6.066e+00, dis_loss=4.451e-03
INFO: Iter 14900: gen_loss=6.516e+00, dis_loss=3.142e-03
INFO: Iter 15000: gen_loss=6.421e+00, dis_loss=2.947e-03
INFO: Iter 15100: gen_loss=6.900e+00, dis_loss=1.797e-03
INFO: Iter 15200: gen_loss=6.958e+00, dis_loss=1.652e-03
INFO: Iter 15300: gen_loss=7.257e+00, dis_loss=1.328e-03
INFO: Iter 15400: gen_loss=7.612e+00, dis_loss=1.029e-03
INFO: Iter 15500: gen_loss=8.009e+00, dis_loss=7.048e-04
INFO: Iter 15600: gen_loss=8.004e+00, dis_loss=1.333e-03
INFO: Iter 15700: gen_loss=8.041e+00, dis_loss=7.921e-04
INFO: Iter 15800: gen_loss=8.045e+00, dis_loss=5.921e-04
INFO: Iter 15900: gen_loss=8.068e+00, dis_loss=5.288e-04
INFO: Iter 16000: gen_loss=7.622e+00, dis_loss=8.960e-04
INFO: Iter 16100: gen_loss=7.309e+00, dis_loss=1.108e-03
INFO: Iter 16200: gen_loss=7.61

INFO: Iter 28900: gen_loss=9.725e+00, dis_loss=1.250e-04
INFO: Iter 29000: gen_loss=9.988e+00, dis_loss=8.539e-05
INFO: Iter 29100: gen_loss=1.014e+01, dis_loss=8.080e-05
INFO: Iter 29200: gen_loss=9.597e+00, dis_loss=1.977e-04
INFO: Iter 29300: gen_loss=1.006e+01, dis_loss=8.565e-05
INFO: Iter 29400: gen_loss=1.025e+01, dis_loss=5.755e-05
INFO: Iter 29500: gen_loss=1.029e+01, dis_loss=6.203e-05
INFO: Iter 29600: gen_loss=1.030e+01, dis_loss=6.991e-05
INFO: Iter 29700: gen_loss=1.021e+01, dis_loss=7.967e-05
INFO: Iter 29800: gen_loss=1.027e+01, dis_loss=6.429e-05
INFO: Iter 29900: gen_loss=1.047e+01, dis_loss=5.714e-05
INFO: Iter 30000: gen_loss=1.067e+01, dis_loss=6.207e-05
INFO: Iter 30100: gen_loss=1.064e+01, dis_loss=6.098e-05
INFO: Iter 30200: gen_loss=1.084e+01, dis_loss=4.322e-05
INFO: Iter 30300: gen_loss=1.099e+01, dis_loss=4.058e-05
INFO: Iter 30400: gen_loss=1.114e+01, dis_loss=3.691e-05
INFO: Iter 30500: gen_loss=1.128e+01, dis_loss=2.777e-05
INFO: Iter 30600: gen_loss=1.13

INFO: Iter 43300: gen_loss=1.007e+01, dis_loss=6.976e-05
INFO: Iter 43400: gen_loss=1.013e+01, dis_loss=5.988e-05
INFO: Iter 43500: gen_loss=9.874e+00, dis_loss=9.952e-05
INFO: Iter 43600: gen_loss=9.436e+00, dis_loss=1.448e-03
INFO: Iter 43700: gen_loss=9.089e+00, dis_loss=1.887e-04
INFO: Iter 43800: gen_loss=8.722e+00, dis_loss=2.714e-04
INFO: Iter 43900: gen_loss=8.790e+00, dis_loss=3.003e-04
INFO: Iter 44000: gen_loss=8.833e+00, dis_loss=4.406e-04
INFO: Iter 44100: gen_loss=9.332e+00, dis_loss=3.817e-04
INFO: Iter 44200: gen_loss=1.032e+01, dis_loss=5.504e-05
INFO: Iter 44300: gen_loss=1.061e+01, dis_loss=5.572e-05
INFO: Iter 44400: gen_loss=1.075e+01, dis_loss=4.529e-05
INFO: Iter 44500: gen_loss=8.709e+00, dis_loss=4.174e-03
INFO: Iter 44600: gen_loss=8.278e+00, dis_loss=4.278e-04
INFO: Iter 44700: gen_loss=8.218e+00, dis_loss=4.829e-04
INFO: Iter 44800: gen_loss=7.334e+00, dis_loss=2.718e-02
INFO: Iter 44900: gen_loss=7.364e+00, dis_loss=1.681e-03
INFO: Iter 45000: gen_loss=7.87

INFO: Iter 57700: gen_loss=1.262e+01, dis_loss=5.628e-06
INFO: Iter 57800: gen_loss=1.287e+01, dis_loss=5.684e-06
INFO: Iter 57900: gen_loss=1.296e+01, dis_loss=6.036e-06
INFO: Iter 58000: gen_loss=1.296e+01, dis_loss=7.623e-06
INFO: Iter 58100: gen_loss=1.307e+01, dis_loss=3.708e-06
INFO: Iter 58200: gen_loss=1.320e+01, dis_loss=3.698e-06
INFO: Iter 58300: gen_loss=1.329e+01, dis_loss=4.310e-06
INFO: Iter 58400: gen_loss=1.341e+01, dis_loss=3.613e-06
INFO: Iter 58500: gen_loss=1.350e+01, dis_loss=4.383e-06
INFO: Iter 58600: gen_loss=1.357e+01, dis_loss=5.090e-06
INFO: Iter 58700: gen_loss=1.368e+01, dis_loss=3.395e-06
INFO: Iter 58800: gen_loss=1.374e+01, dis_loss=3.276e-06
INFO: Iter 58900: gen_loss=1.384e+01, dis_loss=2.718e-06
INFO: Iter 59000: gen_loss=1.390e+01, dis_loss=2.273e-06
INFO: Iter 59100: gen_loss=1.400e+01, dis_loss=2.221e-06
INFO: Iter 59200: gen_loss=1.408e+01, dis_loss=2.595e-06
INFO: Iter 59300: gen_loss=1.415e+01, dis_loss=1.784e-06
INFO: Iter 59400: gen_loss=1.42

...

INFO: Iter 711500: gen_loss=8.466e+01, dis_loss=0.000e+00
INFO: Iter 711600: gen_loss=8.463e+01, dis_loss=0.000e+00
INFO: Iter 711700: gen_loss=8.400e+01, dis_loss=0.000e+00
INFO: Iter 711800: gen_loss=8.453e+01, dis_loss=0.000e+00
INFO: Iter 711900: gen_loss=8.508e+01, dis_loss=0.000e+00
INFO: Iter 712000: gen_loss=8.411e+01, dis_loss=0.000e+00
INFO: Iter 712100: gen_loss=8.495e+01, dis_loss=0.000e+00
INFO: Iter 712200: gen_loss=8.458e+01, dis_loss=0.000e+00
INFO: Iter 712300: gen_loss=8.458e+01, dis_loss=0.000e+00
INFO: Iter 712400: gen_loss=8.453e+01, dis_loss=0.000e+00
INFO: Iter 712500: gen_loss=8.487e+01, dis_loss=0.000e+00
INFO: Iter 712600: gen_loss=8.433e+01, dis_loss=0.000e+00
INFO: Iter 712700: gen_loss=8.432e+01, dis_loss=0.000e+00
INFO: Iter 712800: gen_loss=8.431e+01, dis_loss=0.000e+00
INFO: Iter 712900: gen_loss=8.546e+01, dis_loss=0.000e+00
INFO: Iter 713000: gen_loss=8.438e+01, dis_loss=0.000e+00
INFO: Iter 713100: gen_loss=8.447e+01, dis_loss=0.000e+00
INFO: Iter 713

INFO: Iter 725700: gen_loss=8.470e+01, dis_loss=0.000e+00
INFO: Iter 725800: gen_loss=8.466e+01, dis_loss=0.000e+00
INFO: Iter 725900: gen_loss=8.424e+01, dis_loss=0.000e+00
INFO: Iter 726000: gen_loss=8.418e+01, dis_loss=0.000e+00
INFO: Iter 726100: gen_loss=8.418e+01, dis_loss=0.000e+00
INFO: Iter 726200: gen_loss=8.428e+01, dis_loss=0.000e+00
INFO: Iter 726300: gen_loss=8.433e+01, dis_loss=0.000e+00
INFO: Iter 726400: gen_loss=8.390e+01, dis_loss=0.000e+00
INFO: Iter 726500: gen_loss=8.481e+01, dis_loss=0.000e+00
INFO: Iter 726600: gen_loss=8.423e+01, dis_loss=0.000e+00
INFO: Iter 726700: gen_loss=8.511e+01, dis_loss=0.000e+00
INFO: Iter 726800: gen_loss=8.499e+01, dis_loss=0.000e+00
INFO: Iter 726900: gen_loss=8.481e+01, dis_loss=0.000e+00
INFO: Iter 727000: gen_loss=8.433e+01, dis_loss=0.000e+00
INFO: Iter 727100: gen_loss=8.480e+01, dis_loss=0.000e+00
INFO: Iter 727200: gen_loss=8.436e+01, dis_loss=0.000e+00
INFO: Iter 727300: gen_loss=8.492e+01, dis_loss=0.000e+00
INFO: Iter 727

INFO: Iter 739900: gen_loss=8.472e+01, dis_loss=0.000e+00
INFO: Iter 740000: gen_loss=8.481e+01, dis_loss=0.000e+00
INFO: Iter 740100: gen_loss=8.475e+01, dis_loss=0.000e+00
INFO: Iter 740200: gen_loss=8.531e+01, dis_loss=0.000e+00
INFO: Iter 740300: gen_loss=8.447e+01, dis_loss=0.000e+00
INFO: Iter 740400: gen_loss=8.459e+01, dis_loss=0.000e+00
INFO: Iter 740500: gen_loss=8.472e+01, dis_loss=0.000e+00
INFO: Iter 740600: gen_loss=8.467e+01, dis_loss=0.000e+00
INFO: Iter 740700: gen_loss=8.483e+01, dis_loss=0.000e+00
INFO: Iter 740800: gen_loss=8.506e+01, dis_loss=0.000e+00
INFO: Iter 740900: gen_loss=8.454e+01, dis_loss=0.000e+00
INFO: Iter 741000: gen_loss=8.534e+01, dis_loss=0.000e+00
INFO: Iter 741100: gen_loss=8.557e+01, dis_loss=0.000e+00
INFO: Iter 741200: gen_loss=8.571e+01, dis_loss=0.000e+00
INFO: Iter 741300: gen_loss=8.501e+01, dis_loss=0.000e+00
INFO: Iter 741400: gen_loss=8.471e+01, dis_loss=0.000e+00
INFO: Iter 741500: gen_loss=8.446e+01, dis_loss=0.000e+00
INFO: Iter 741

INFO: Iter 754100: gen_loss=8.527e+01, dis_loss=0.000e+00
INFO: Iter 754200: gen_loss=8.498e+01, dis_loss=0.000e+00
INFO: Iter 754300: gen_loss=8.606e+01, dis_loss=0.000e+00
INFO: Iter 754400: gen_loss=8.481e+01, dis_loss=0.000e+00
INFO: Iter 754500: gen_loss=8.450e+01, dis_loss=0.000e+00
INFO: Iter 754600: gen_loss=8.501e+01, dis_loss=0.000e+00
INFO: Iter 754700: gen_loss=8.460e+01, dis_loss=0.000e+00
INFO: Iter 754800: gen_loss=8.473e+01, dis_loss=0.000e+00
INFO: Iter 754900: gen_loss=8.470e+01, dis_loss=0.000e+00
INFO: Iter 755000: gen_loss=8.460e+01, dis_loss=0.000e+00
INFO: Iter 755100: gen_loss=8.518e+01, dis_loss=0.000e+00
INFO: Iter 755200: gen_loss=8.486e+01, dis_loss=0.000e+00
INFO: Iter 755300: gen_loss=8.495e+01, dis_loss=0.000e+00
INFO: Iter 755400: gen_loss=8.541e+01, dis_loss=0.000e+00
INFO: Iter 755500: gen_loss=8.498e+01, dis_loss=0.000e+00
INFO: Iter 755600: gen_loss=8.497e+01, dis_loss=0.000e+00
INFO: Iter 755700: gen_loss=8.496e+01, dis_loss=0.000e+00
INFO: Iter 755

KeyboardInterrupt: 

In [8]:
%%bash

echo "Starting tensorboard"
tensorboard --logdir runs --host 0.0.0.0


Process is interrupted.
